# Efficient Adaptation and Analysis of Vision Transformers


In [ ]:
import torch
import sys
print("Imported torch and sys")


Imported torch and sys


In [2]:
!nvidia-smi

Sat Dec  6 22:21:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P0             26W /   70W |     105MiB /  15360MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install wandb
# !wandb login

import wandb
from kaggle_secrets import UserSecretsClient

# Get the secret value
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# Log in using the key
wandb.login(key=wandb_api_key)

print("Loading wandb screts to log into W&B!")

Loading wandb screts to log into W&B!


#### Downloading the data

In [4]:
import torchvision
import torchvision.transforms as transforms

# Define a transform to normalize the data
# You'll likely want to resize to 224x224 and use ImageNet stats for ViT
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# Load the Food-101 training dataset
# Food-101 uses 'split="train"' for training
trainset = torchvision.datasets.Food101(root='./data', split="train",
                                        download=True, transform=transform)

# Load the Food-101 test (validation) dataset
# Food-101 uses 'split="test"' for testing
testset = torchvision.datasets.Food101(root='./data', split="test",
                                       download=True, transform=transform)

print("Food-101 dataset imported successfully.")
print(f"Training set size: {len(trainset)}")
print(f"Test set size: {len(testset)}")

100%|██████████| 5.00G/5.00G [02:51<00:00, 29.2MB/s] 


Food-101 dataset imported successfully.
Training set size: 75750
Test set size: 25250


#### Resizing the data for ViT model


In [5]:
import torchvision.transforms as transforms

# Define transforms for training and validation/testing
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Upsample to ViT resolution
    transforms.RandomHorizontalFlip(), # Example data augmentation
    transforms.RandomCrop(224, padding=4), # Example data augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)), # Upsample to ViT resolution
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Apply the transforms to the datasets
trainset.transform = train_transform
testset.transform = test_transform

print("Data preparation complete. Transforms applied to datasets.")

Data preparation complete. Transforms applied to datasets.


#### Loading the VIT and freezing the parameters

In [6]:
from transformers import ViTForImageClassification

# Load a pre-trained ViT-Huge model
model = ViTForImageClassification.from_pretrained(
    'google/vit-huge-patch14-224-in21k',  # Model name for ViT-Huge
    num_labels=101,                       # Updated for 101 classes in Food-101
    ignore_mismatched_sizes=True          # Allows replacing the classifier head
)

print("Pre-trained ViT-Huge model loaded.")

2025-12-06 22:29:47.900245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765060188.057582      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765060188.102737      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.53G [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-huge-patch14-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pre-trained ViT-Huge model loaded.


# Here we decide if we want to train full model or some

In [7]:
# Print the number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")

Number of trainable parameters: 630894181


In [8]:
model

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-31): 32 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=1280, out_features=1280, bias=True)
              (key): Linear(in_features=1280, out_features=1280, bias=True)
              (value): Linear(in_features=1280, out_features=1280, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=1280, out_features=1280, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=1280, out_features=5120, bias=True)
           

## Step 1: Create the DataLoaders

In [9]:
from torch.utils.data import DataLoader

# Create DataLoaders
train_loader = DataLoader(trainset, batch_size=16, shuffle=True)
test_loader = DataLoader(testset, batch_size=16, shuffle=False)

print("DataLoaders created.")

DataLoaders created.


## Step 2: Enable Gradient Checkpointing

In [10]:
# Enable gradient checkpointing
model.gradient_checkpointing_enable()
print("Gradient checkpointing enabled.")

Gradient checkpointing enabled.


## Step 3: Initialize DeepSpeed

In [11]:
%pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.2 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 988.2 kB/s eta 0:00:000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00

In [13]:
!apt-get install -y libaio-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libaio-dev
0 upgraded, 1 newly installed, 0 to remove and 165 not upgraded.
Need to get 21.2 kB of archives.
After this operation, 71.7 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libaio-dev amd64 0.3.112-13build1 [21.2 kB]
Fetched 21.2 kB in 0s (193 kB/s)      
Selecting previously unselected package libaio-dev:amd64.
(Reading database ... 128639 files and directories currently installed.)
Preparing to unpack .../libaio-dev_0.3.112-13build1_amd64.deb ...
Unpacking libaio-dev:amd64 (0.3.112-13build1) ...
Setting up libaio-dev:amd64 (0.3.112-13build1) ...
Processing triggers for man-db (2.10.2-1) ...


## Script for Standard Full Fine-Tune Using DDP, running on 2 T4 gpus batch size = 16 per gpu

In [14]:
%%writefile train_standard_ddp_vit_huge_bs16.py
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTImageProcessor
import warnings
import wandb
import os
import time
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
from torch.cuda.amp import GradScaler, autocast
import torch.profiler 
os.makedirs("./profiling_traces", exist_ok=True)

# --- 1. Setup & Helper Functions ---
warnings.filterwarnings("ignore")
print("--- Initializing Standard DDP ---")

rank = int(os.environ['RANK'])
local_rank = int(os.environ['LOCAL_RANK'])
world_size = int(os.environ['WORLD_SIZE'])

dist.init_process_group(backend='nccl')
torch.cuda.set_device(local_rank)
device = torch.device(f"cuda:{local_rank}")

def inspect_model_sharding(model, name_tag):
    """Checks if parameters are sharded (Expected: NO for DDP)"""
    if rank == 0:
        print(f"\n[Rank {rank}] --- INSPECTING SHARDING ({name_tag}) ---")
        # Unwrap DDP to get to the actual model parameters
        actual_model = model.module if hasattr(model, "module") else model
        
        count = 0
        for name, param in actual_model.named_parameters():
            print(f"Param: {name}")
            # In DDP, Physical Shape should equal Logical Shape (Full Layer)
            print(f"  Physical Shape: {param.shape} (Elements: {param.numel():,})")
            print(f"  Status:         REPLICATED (Full Copy)")
            print("-" * 40)
            count += 1
            if count >= 2: break
        print("----------------------------------\n")

# --- 2. Data & Model ---
if rank == 0: print("Setting up Data & Model...")
processor = ViTImageProcessor.from_pretrained('google/vit-huge-patch14-224-in21k')
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

if rank == 0:
    trainset = torchvision.datasets.Food101(root='./data', split="train", download=True, transform=train_transform)
dist.barrier()
if rank != 0:
    trainset = torchvision.datasets.Food101(root='./data', split="train", download=False, transform=train_transform)

train_sampler = DistributedSampler(trainset, num_replicas=world_size, rank=rank, shuffle=True)
train_loader = DataLoader(trainset, batch_size=16, sampler=train_sampler, num_workers=2, pin_memory=True)

model = ViTForImageClassification.from_pretrained('google/vit-huge-patch14-224-in21k', num_labels=101, ignore_mismatched_sizes=True)
model.gradient_checkpointing_enable()
model.to(device)
model = DDP(model, device_ids=[local_rank], output_device=local_rank)

optimizer = optim.AdamW(model.parameters(), lr=5e-5)
scaler = GradScaler(enabled=True)

# --- 3. Logging & Inspection ---
if rank == 0:
    wandb.init(
        project="Distributed ViT training systems-Latest_run",
        name="Standard-DDP-ViT-Huge-Food101-2xT4-Batch-size-16-per-gpu",
        config={"model": "vit-huge", "mode": "DDP"}
    )

# PRO CHECK: This output will prove that DDP duplicates the full model on every GPU
inspect_model_sharding(model, "Standard DDP")

# --- 4. Training Loop with Labels ---
print(f"[Rank {rank}] Starting training...")

for i, batch in enumerate(train_loader):
    # Stop early to save time (we just need traces and logs)
    if i >= 6: break
    
    # LABEL 0: Top Level Step
    with torch.profiler.record_function(f"## Training Step {i} ##"):
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad(set_to_none=True)

        # LABEL 1: Forward
        with torch.profiler.record_function("## Forward Pass ##"):
            with autocast(enabled=True):
                outputs = model(inputs, labels=labels)
                loss = outputs.loss

        # LABEL 2: Backward
        with torch.profiler.record_function("## Backward Pass ##"):
            # This is where it usually crashes (OOM)
            scaler.scale(loss).backward()

        # LABEL 3: Optimizer
        with torch.profiler.record_function("## Optimizer Step ##"):
            scaler.step(optimizer)
            scaler.update()

        # --- PRO LOGGING ---
        if rank == 0:
            mem = torch.cuda.memory_allocated() / 1e9
            max_mem = torch.cuda.max_memory_allocated() / 1e9
            wandb.log({
                "loss": loss.item(),
                "System/Memory_Allocated_GB": mem,
                "System/Max_Memory_GB": max_mem
            })
            print(f"Step {i}: Loss={loss.item():.4f} | Mem={mem:.2f}GB")

dist.destroy_process_group()

Writing train_standard_ddp_vit_huge_bs16.py


In [15]:
# 1. Install/Login
!pip install -q transformers datasets peft deepspeed wandb scikit-learn seaborn tensorboard
!wandb login

# 2. Launch the DDP script
# !torchrun --nproc_per_node=2 train_standard_ddp.py
!torchrun --nproc_per_node=2 --master_port 29501 train_standard_ddp_vit_huge_bs16.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 41.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False wa

## Script for DeepSpeed Full Fine-Tune, running on 2 T4 gpus stage 2

In [16]:
%%writefile deepspeed_config_stage2_vit_huge_bs16.json
{
    "fp16": {
        "enabled": true,
        "loss_scale": 0,
        "loss_scale_window": 1000
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": 5e-5,
            "betas": [0.9, 0.999],
            "eps": 1e-8,
            "weight_decay": 3e-7
        }
    },
    "zero_optimization": {
        "stage": 2,
        "allgather_partitions": true,
        "allgather_bucket_size": 5e8,
        "reduce_scatter": true,
        "reduce_bucket_size": 5e8,
        "overlap_comm": true,
        "contiguous_gradients": true
    },
    "train_batch_size": 32,
    "train_micro_batch_size_per_gpu": 16,
    "gradient_accumulation_steps": 1,
    "gradient_clipping": 1.0,
    "steps_per_print": 1,
    "wall_clock_breakdown": false,
    "flops_profiler": {
        "enabled": false
    }
}

Writing deepspeed_config_stage2_vit_huge_bs16.json


In [17]:
%%writefile train_deepspeed_stage2_vit_huge_bs16.py
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTImageProcessor
import deepspeed
import warnings
import wandb
import os
import shutil
import time
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
import torch.profiler
import json
import glob

# --- TRACE DIRECTORY SETUP ---
TRACE_DIR = "./pytorch_traces/deepspeed_stage2_trace"
MERGED_TRACE_FILE = "./pytorch_traces/deepspeed_stage2_trace/merged_multi_gpu_trace.json"

# Cleanup old runs to avoid confusion (only rank 0)
if int(os.environ.get('RANK', 0)) == 0:
    if os.path.exists(TRACE_DIR):
        try: 
            shutil.rmtree(TRACE_DIR)
        except: 
            pass
    os.makedirs(TRACE_DIR, exist_ok=True)
    if os.path.exists(MERGED_TRACE_FILE):
        os.remove(MERGED_TRACE_FILE)

# Suppress warnings
warnings.filterwarnings("ignore")
print("--- Initializing DEEPSPEED ZeRO Stage 2 (Multi-GPU Profiling) ---")

# --- 1. Setup ---
local_rank = int(os.environ.get('LOCAL_RANK', 0))
rank = int(os.environ.get('RANK', 0))
world_size = int(os.environ.get('WORLD_SIZE', 1))

if torch.cuda.is_available():
    torch.cuda.set_device(local_rank)
    print(f"[Rank {rank}] Initializing process on GPU {local_rank}...")
else:
    print(f"[Rank {rank}] WARNING: No CUDA available, running on CPU")

# --- W&B Setup ---
if rank == 0:
    wandb.init(
        project="Distributed ViT training systems-Latest_run", 
        name="DeepSpeed-Stage2-ViT-Huge-Food101-2xT4-MBS16-GAS1-Batch-size-16-per-gpu",
        config={
            "optimization": "Stage 2",
            "num_gpus": world_size,
            "profiling": "Rank 0 & 1 (Merged)"
        }
    )

# --- 2. Model Setup ---
if rank == 0: print("Loading pre-trained ViT-Huge model...")
model = ViTForImageClassification.from_pretrained(
    'google/vit-huge-patch14-224-in21k', 
    num_labels=101, 
    ignore_mismatched_sizes=True
)
model.gradient_checkpointing_enable()

# --- 3. DeepSpeed Init ---
if rank == 0: print("Initializing DeepSpeed...")
model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model, 
    model_parameters=model.parameters(), 
    config_params='deepspeed_config_stage2_vit_huge_bs16.json'
)

# --- 4. Data Prep ---
if rank == 0: print("Setting up data...")
processor = ViTImageProcessor.from_pretrained('google/vit-huge-patch14-224-in21k')
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

if rank == 0:
    trainset = torchvision.datasets.Food101(root='./data', split="train", download=True, transform=train_transform)

if world_size > 1: dist.barrier()
    
if rank != 0:
    trainset = torchvision.datasets.Food101(root='./data', split="train", download=False, transform=train_transform)

MICRO_BATCH_SIZE = 16
train_sampler = DistributedSampler(trainset, num_replicas=world_size, rank=rank, shuffle=True)
train_loader = DataLoader(trainset, batch_size=MICRO_BATCH_SIZE, sampler=train_sampler, num_workers=2, pin_memory=True)

# --- ROBUST MERGE FUNCTION ---
def merge_traces_logic():
    """Merge all rank traces into a single file for Perfetto (Handles String PIDs)"""
    print(f"\n[Rank 0] Merging traces from all ranks...")
    trace_files = sorted(glob.glob(os.path.join(TRACE_DIR, "*.pt.trace.json")))
    
    if not trace_files:
        print("  WARNING: No trace files found yet!")
        return
    
    merged_events = []
    metadata = {}
    
    for idx, trace_file in enumerate(trace_files):
        try:
            with open(trace_file, 'r') as f:
                data = json.load(f)
            
            if isinstance(data, dict):
                events = data.get("traceEvents", [])
                if idx == 0: metadata = {k: v for k, v in data.items() if k != "traceEvents"}
            else:
                events = data
            
            # --- ROBUST PID/TID OFFSETTING ---
            for event in events:
                # 1. Safely Offset PID
                if "pid" in event:
                    try:
                        event["pid"] = int(event["pid"]) + (idx * 100000)
                    except ValueError:
                        event["pid"] = f"{event['pid']}_{idx}"

                # 2. Safely Offset TID
                if "tid" in event:
                    try:
                        event["tid"] = int(event["tid"]) + (idx * 100000)
                    except ValueError:
                        event["tid"] = f"{event['tid']}_{idx}"
                
                # 3. Rename
                if "name" in event and isinstance(event["name"], str):
                    if not event["name"].startswith("[GPU"):
                        event["name"] = f"[GPU {idx}] {event['name']}"
            
            merged_events.extend(events)
            print(f"    Added {len(events)} events from GPU {idx}")
        except Exception as e:
            print(f"    Warning: Could not read {trace_file}: {e}")

    merged_trace = metadata.copy()
    merged_trace["traceEvents"] = merged_events
    
    with open(MERGED_TRACE_FILE, 'w') as f:
        json.dump(merged_trace, f)
    
    print(f"\n  MERGE COMPLETE!")
    print(f"  Saved to: {MERGED_TRACE_FILE}")
    print(f" Upload this file to https://ui.perfetto.dev/\n")

# --- Training Configuration ---
device = model_engine.device
num_epochs = 1
start_time = time.time()
PROFILING_STEPS = 0

if rank == 0: 
    print(f"\n{'='*70}")
    print(f"--- STARTING TRAINING WITH PROFILING ---")
    print(f"  - Profiling: ALL ranks (first {PROFILING_STEPS} steps)")
    print(f"  - Training: Continues FULLY after profiling")
    print(f"{'='*70}\n")

try:
    # --- PROFILER INITIALIZATION (ALL RANKS) ---
    prof = torch.profiler.profile(
        activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
        schedule=torch.profiler.schedule(wait=1, warmup=2, active=5, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler(TRACE_DIR),
        record_shapes=True,
        profile_memory=True,
        with_stack=True,
        with_flops=True
    )
    # prof.start()
    
    model_engine.train()
    
    for epoch in range(num_epochs):
        train_sampler.set_epoch(epoch)
        epoch_loss = 0.0
        num_batches = 0
        
        for i, batch in enumerate(train_loader):
            
            # --- Profiling Window ---
            is_profiling = (i < PROFILING_STEPS)
            prof_context = None
            
            if is_profiling:
                prof_context = torch.profiler.record_function(f"## [GPU {rank}] Step {i} ##")
                prof_context.__enter__()
            
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward
            if is_profiling:
                with torch.profiler.record_function(f"## [GPU {rank}] FWD ##"):
                    outputs = model_engine(inputs, labels=labels)
                    loss = outputs.loss
            else:
                outputs = model_engine(inputs, labels=labels)
                loss = outputs.loss

            # Backward
            if is_profiling:
                with torch.profiler.record_function(f"## [GPU {rank}] BWD ##"):
                    model_engine.backward(loss)
            else:
                model_engine.backward(loss)

            # Step
            if is_profiling:
                with torch.profiler.record_function(f"## [GPU {rank}] OPT ##"):
                    model_engine.step()
            else:
                model_engine.step()

            # Metrics
            loss_val = loss.item()
            epoch_loss += loss_val
            num_batches += 1

            if rank == 0:
                wandb.log({"loss": loss_val, "step": i})
                if i % 10 == 0:
                    print(f"  [GPU {rank}] Epoch {epoch} | Step {i} | Loss: {loss_val:.4f}")

            # --- Profiler Stepping ---
            if is_profiling:
                prof.step()
                if prof_context: prof_context.__exit__(None, None, None)
            
            # --- STOP & MERGE (But continue training) ---
            if i == PROFILING_STEPS:
                prof.stop() # Save trace files to disk
                
                # 1. Sync all ranks to ensure writing is done
                if world_size > 1: dist.barrier()
                
                # 2. Rank 0 performs the merge immediately
                if rank == 0:
                    print(f"\n>>> Profiling complete. Merging traces while training continues... <<<")
                    merge_traces_logic()
                    print(">>> Resuming full training loop... <<<\n")

    # End of Epoch
    if rank == 0:
        print(f"\n{'='*70}")
        print(f"TRAINING COMPLETE")
        print(f"{'='*70}\n")
        wandb.finish()

except Exception as e:
    print(f"[Rank {rank}] ERROR: {e}")
    import traceback
    traceback.print_exc()
finally:
    if world_size > 1:
        dist.barrier()
        dist.destroy_process_group()
    print(f"--- [Rank {rank}] Done ---")

Writing train_deepspeed_stage2_vit_huge_bs16.py


In [18]:
!deepspeed --num_gpus=2 train_deepspeed_stage2_vit_huge_bs16.py

2025-12-06 22:37:27.136778: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765060647.159408     406 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765060647.166145     406 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
[2025-12-06 22:37:32,429] [WARNING] [runner.py:232:fetch_hostfile] Unable to 

In [20]:
import os
os.system('zip -r deepspeed_stage2_trace.zip /kaggle/working/pytorch_traces/deepspeed_stage2_trace')


  adding: kaggle/working/pytorch_traces/deepspeed_stage2_trace/ (stored 0%)
  adding: kaggle/working/pytorch_traces/deepspeed_stage2_trace/merged_multi_gpu_trace.json (deflated 93%)
  adding: kaggle/working/pytorch_traces/deepspeed_stage2_trace/e9b764cef546_490.1764732007835768228.pt.trace.json (deflated 93%)
  adding: kaggle/working/pytorch_traces/deepspeed_stage2_trace/e9b764cef546_491.1764732007918481049.pt.trace.json (deflated 93%)


0

In [28]:
from pathlib import Path

path = Path('/kaggle/working/pytorch_traces/deepspeed_stage3_trace')

if path.is_file():
    path.unlink()  # Delete file
elif path.is_dir():
    import shutil
    shutil.rmtree(path)

## Script for DeepSpeed Full Fine-Tune, running on 2 T4 gpus stage 3

In [19]:
%%writefile deepspeed_config_stage3_vit_huge_bs16.json
{
    "fp16": {
        "enabled": true,
        "loss_scale": 0,
        "loss_scale_window": 1000
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": 5e-5,
            "betas": [0.9, 0.999],
            "eps": 1e-8,
            "weight_decay": 3e-7
        }
    },
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "none"
        },
        "offload_param": {
            "device": "none"
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "stage3_max_live_parameters": 1e9,
        "stage3_max_reuse_distance": 1e9,
        "stage3_prefetch_bucket_size": 5e7,
        "stage3_param_persistence_threshold": 1e5,
        "reduce_bucket_size": 5e8,
        "sub_group_size": 1e9
    },
    "train_batch_size": 32,
    "train_micro_batch_size_per_gpu": 16,
    "gradient_accumulation_steps": 1,
    "gradient_clipping": 1.0,
    "steps_per_print": 1,
    "wall_clock_breakdown": false,
    "flops_profiler": {
        "enabled": false
    }
}

Writing deepspeed_config_stage3_vit_huge_bs16.json


In [20]:
%%writefile train_deepspeed_stage3_vit_huge_bs16.py
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTImageProcessor
import deepspeed
import warnings
import wandb
import os
import shutil
import time
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
import torch.profiler
import json
import glob

# --- TRACE DIRECTORY SETUP ---
TRACE_DIR = "./pytorch_traces/deepspeed_stage3_trace"
MERGED_TRACE_FILE = "./pytorch_traces/deepspeed_stage3_trace/merged_stage3_trace.json"

# Cleanup old runs to avoid confusion (only rank 0)
if int(os.environ.get('RANK', 0)) == 0:
    if os.path.exists(TRACE_DIR):
        try: shutil.rmtree(TRACE_DIR)
        except: pass
    os.makedirs(TRACE_DIR, exist_ok=True)
    if os.path.exists(MERGED_TRACE_FILE):
        os.remove(MERGED_TRACE_FILE)

# Suppress warnings
warnings.filterwarnings("ignore")
print("--- Initializing DEEPSPEED ZeRO Stage 3 (Multi-GPU Profiling) ---")

# --- 1. Setup ---
local_rank = int(os.environ.get('LOCAL_RANK', 0))
rank = int(os.environ.get('RANK', 0))
world_size = int(os.environ.get('WORLD_SIZE', 1))

if torch.cuda.is_available():
    torch.cuda.set_device(local_rank)
    print(f"[Rank {rank}] Initializing process on GPU {local_rank}...")
else:
    print(f"[Rank {rank}] WARNING: No CUDA available, running on CPU")

# --- W&B Setup ---
if rank == 0:
    wandb.init(
        project="Distributed ViT training systems-Latest_run", 
        name="DeepSpeed-Stage3-ViT-Huge-Food101-2xT4-MBS16-GAS1-Batch-size-16-per-gpu",
        config={
            "optimization": "Stage 3",
            "num_gpus": world_size,
            "profiling": "Rank 0 & 1 (Merged)"
        }
    )

# --- 2. Model Setup ---
if rank == 0: print("Loading pre-trained ViT-Huge model...")
model = ViTForImageClassification.from_pretrained(
    'google/vit-huge-patch14-224-in21k', 
    num_labels=101, 
    ignore_mismatched_sizes=True
)
model.gradient_checkpointing_enable()

# --- 3. DeepSpeed Init ---
if rank == 0: print("Initializing DeepSpeed...")
model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model, 
    model_parameters=model.parameters(), 
    config_params='deepspeed_config_stage3_vit_huge_bs16.json'
)

# --- 4. Data Prep ---
if rank == 0: print("Setting up data...")
processor = ViTImageProcessor.from_pretrained('google/vit-huge-patch14-224-in21k')
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

if rank == 0:
    trainset = torchvision.datasets.Food101(root='./data', split="train", download=True, transform=train_transform)

if world_size > 1: dist.barrier()
    
if rank != 0:
    trainset = torchvision.datasets.Food101(root='./data', split="train", download=False, transform=train_transform)

MICRO_BATCH_SIZE = 16
train_sampler = DistributedSampler(trainset, num_replicas=world_size, rank=rank, shuffle=True)
train_loader = DataLoader(trainset, batch_size=MICRO_BATCH_SIZE, sampler=train_sampler, num_workers=2, pin_memory=True)

# --- ROBUST MERGE FUNCTION ---
def merge_traces_logic():
    """Merge all rank traces into a single file for Perfetto (Handles String PIDs)"""
    print(f"\n[Rank 0] Merging traces from all ranks...")
    trace_files = sorted(glob.glob(os.path.join(TRACE_DIR, "*.pt.trace.json")))
    
    if not trace_files:
        print("  WARNING: No trace files found yet!")
        return
    
    merged_events = []
    metadata = {}
    
    for idx, trace_file in enumerate(trace_files):
        try:
            with open(trace_file, 'r') as f:
                data = json.load(f)
            
            if isinstance(data, dict):
                events = data.get("traceEvents", [])
                if idx == 0: metadata = {k: v for k, v in data.items() if k != "traceEvents"}
            else:
                events = data
            
            # --- ROBUST PID/TID OFFSETTING ---
            for event in events:
                if "pid" in event:
                    try: event["pid"] = int(event["pid"]) + (idx * 100000)
                    except ValueError: event["pid"] = f"{event['pid']}_{idx}"

                if "tid" in event:
                    try: event["tid"] = int(event["tid"]) + (idx * 100000)
                    except ValueError: event["tid"] = f"{event['tid']}_{idx}"
                
                if "name" in event and isinstance(event["name"], str):
                    if not event["name"].startswith("[GPU"):
                        event["name"] = f"[GPU {idx}] {event['name']}"
            
            merged_events.extend(events)
            print(f"    Added {len(events)} events from GPU {idx}")
        except Exception as e:
            print(f"    Warning: Could not read {trace_file}: {e}")

    merged_trace = metadata.copy()
    merged_trace["traceEvents"] = merged_events
    
    with open(MERGED_TRACE_FILE, 'w') as f:
        json.dump(merged_trace, f)
    
    print(f"\n  MERGE COMPLETE!")
    print(f"  Saved to: {MERGED_TRACE_FILE}")
    print(f"  Upload this file to https://ui.perfetto.dev/\n")

# --- Training Configuration ---
device = model_engine.device
num_epochs = 1
start_time = time.time()
PROFILING_STEPS = 0

if rank == 0: 
    print(f"\n{'='*70}")
    print(f"--- STARTING TRAINING WITH PROFILING ---")
    print(f"  - Profiling: ALL ranks (first {PROFILING_STEPS} steps)")
    print(f"  - Training: Continues FULLY after profiling")
    print(f"{'='*70}\n")

try:
    # --- PROFILER INITIALIZATION (ALL RANKS) ---
    prof = torch.profiler.profile(
        activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
        schedule=torch.profiler.schedule(wait=1, warmup=2, active=5, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler(TRACE_DIR),
        record_shapes=True,
        profile_memory=True,
        with_stack=True,
        with_flops=True
    )
    # prof.start()
    
    model_engine.train()
    
    for epoch in range(num_epochs):
        train_sampler.set_epoch(epoch)
        epoch_loss = 0.0
        num_batches = 0
        
        for i, batch in enumerate(train_loader):
            
            # --- Profiling Window ---
            is_profiling = (i < PROFILING_STEPS)
            prof_context = None
            
            if is_profiling:
                prof_context = torch.profiler.record_function(f"## [GPU {rank}] Step {i} ##")
                prof_context.__enter__()
            
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward
            if is_profiling:
                with torch.profiler.record_function(f"## [GPU {rank}] FWD ##"):
                    outputs = model_engine(inputs, labels=labels)
                    loss = outputs.loss
            else:
                outputs = model_engine(inputs, labels=labels)
                loss = outputs.loss

            # Backward
            if is_profiling:
                with torch.profiler.record_function(f"## [GPU {rank}] BWD ##"):
                    model_engine.backward(loss)
            else:
                model_engine.backward(loss)

            # Step
            if is_profiling:
                with torch.profiler.record_function(f"## [GPU {rank}] OPT ##"):
                    model_engine.step()
            else:
                model_engine.step()

            # Metrics
            loss_val = loss.item()
            epoch_loss += loss_val
            num_batches += 1

            if rank == 0:
                wandb.log({"loss": loss_val, "step": i})
                if i % 10 == 0:
                    print(f"  [GPU {rank}] Epoch {epoch} | Step {i} | Loss: {loss_val:.4f}")

            # --- Profiler Stepping ---
            if is_profiling:
                prof.step()
                if prof_context: prof_context.__exit__(None, None, None)
            
            # --- STOP & MERGE (But continue training) ---
            if i == PROFILING_STEPS:
                prof.stop() # Save trace files to disk
                
                # 1. Sync all ranks to ensure writing is done
                if world_size > 1: dist.barrier()
                
                # 2. Rank 0 performs the merge immediately
                if rank == 0:
                    print(f"\n>>> Profiling complete. Merging traces while training continues... <<<")
                    merge_traces_logic()
                    print(">>> Resuming full training loop... <<<\n")

    # End of Epoch
    if rank == 0:
        print(f"\n{'='*70}")
        print(f"TRAINING COMPLETE")
        print(f"{'='*70}\n")
        wandb.finish()

except Exception as e:
    print(f"[Rank {rank}] ERROR: {e}")
    import traceback
    traceback.print_exc()
finally:
    if world_size > 1:
        dist.barrier()
        dist.destroy_process_group()
    print(f"--- [Rank {rank}] Done ---")

Writing train_deepspeed_stage3_vit_huge_bs16.py


In [21]:
!deepspeed --num_gpus=2 train_deepspeed_stage3_vit_huge_bs16.py

2025-12-07 00:08:24.907472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765066104.932859     736 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765066104.941369     736 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
[2025-12-07 00:08:30,277] [WARNING] [runner.py:232:fetch_hostfile] Unable to 

## Script for DeepSpeed Full Fine-Tune, running on 2 T4 gpus stage 3 Zero ++

In [22]:
%%writefile deepspeed_config_zeropp.json
{
    "fp16": {
        "enabled": true,
        "loss_scale": 0,
        "loss_scale_window": 1000
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": 5e-5,
            "betas": [0.9, 0.999],
            "eps": 1e-8,
            "weight_decay": 3e-7
        }
    },
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "none"
        },
        "offload_param": {
            "device": "none"
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 1e9,
        "reduce_bucket_size": 5e8,
        "stage3_prefetch_bucket_size": 5e7,
        "stage3_param_persistence_threshold": 1e5,
        "stage3_max_live_parameters": 1e9,
        "stage3_max_reuse_distance": 1e9,
        "gather_16bit_weights_on_model_save": true,
        // --- ZeRO++ ---
        "zero_quantized_weights": false,
        "zero_quantized_gradients": true,
        "zero_hpz_partition_size": 1
    },
    "train_batch_size": 32,
    "train_micro_batch_size_per_gpu": 16,
    "gradient_accumulation_steps": 1,
    "gradient_clipping": 1.0,
    "steps_per_print": 1,
    "wall_clock_breakdown": false,
    "flops_profiler": {
        "enabled": false
    }
}

Writing deepspeed_config_zeropp.json


In [ ]:
%%writefile train_deepspeed_zeropp.py
import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTImageProcessor
import deepspeed
import warnings
import wandb
import os
import shutil
import time
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
import torch.profiler
import json
import glob

# --- TRACE DIRECTORY SETUP ---
TRACE_DIR = "./pytorch_traces/deepspeed_zeropp_trace"
MERGED_TRACE_FILE = "./pytorch_traces/deepspeed_zeropp_trace/merged_zeropp_trace.json"

# Cleanup old runs to avoid confusion (only rank 0)
if int(os.environ.get('RANK', 0)) == 0:
    if os.path.exists(TRACE_DIR):
        try: shutil.rmtree(TRACE_DIR)
        except: pass
    os.makedirs(TRACE_DIR, exist_ok=True)
    if os.path.exists(MERGED_TRACE_FILE):
        os.remove(MERGED_TRACE_FILE)

# Suppress warnings
warnings.filterwarnings("ignore")
print("--- Initializing DEEPSPEED ZeRO++ (Quantized Communication) ---")

# --- 1. Setup ---
local_rank = int(os.environ.get('LOCAL_RANK', 0))
rank = int(os.environ.get('RANK', 0))
world_size = int(os.environ.get('WORLD_SIZE', 1))

if torch.cuda.is_available():
    torch.cuda.set_device(local_rank)
    print(f"[Rank {rank}] Initializing process on GPU {local_rank}...")
else:
    print(f"[Rank {rank}] WARNING: No CUDA available, running on CPU")

# --- W&B Setup ---
if rank == 0:
    wandb.init(
        project="Distributed ViT training systems-Latest_run", 
        name="DeepSpeed-ZeRO-PlusPlus-ViT-Huge-Food101-2xT4-MBS16-GAS1-Batch-size-16-per-gpu",
        config={
            "optimization": "ZeRO++",
            "num_gpus": world_size,
            "profiling": "Rank 0 & 1 (Merged)"
        }
    )

# --- 2. Model Setup ---
if rank == 0: print("Loading pre-trained ViT-Huge model...")
model = ViTForImageClassification.from_pretrained(
    'google/vit-huge-patch14-224-in21k', 
    num_labels=101, 
    ignore_mismatched_sizes=True
)
model.gradient_checkpointing_enable()

# --- 3. DeepSpeed Init ---
if rank == 0: print("Initializing DeepSpeed with ZeRO++ Config...")
model_engine, optimizer, _, _ = deepspeed.initialize(
    model=model, 
    model_parameters=model.parameters(), 
    config_params='deepspeed_config_zeropp.json'
)

# --- 4. Data Prep ---
if rank == 0: print("Setting up data...")
processor = ViTImageProcessor.from_pretrained('google/vit-huge-patch14-224-in21k')
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

if rank == 0:
    trainset = torchvision.datasets.Food101(root='./data', split="train", download=True, transform=train_transform)

if world_size > 1: dist.barrier()
    
if rank != 0:
    trainset = torchvision.datasets.Food101(root='./data', split="train", download=False, transform=train_transform)

MICRO_BATCH_SIZE = 16
train_sampler = DistributedSampler(trainset, num_replicas=world_size, rank=rank, shuffle=True)
train_loader = DataLoader(trainset, batch_size=MICRO_BATCH_SIZE, sampler=train_sampler, num_workers=2, pin_memory=True)

# --- ROBUST MERGE FUNCTION ---
def merge_traces_logic():
    """Merge all rank traces into a single file for Perfetto (Handles String PIDs)"""
    print(f"\n[Rank 0] Merging traces from all ranks...")
    trace_files = sorted(glob.glob(os.path.join(TRACE_DIR, "*.pt.trace.json")))
    
    if not trace_files:
        print("  WARNING: No trace files found yet!")
        return
    
    merged_events = []
    metadata = {}
    
    for idx, trace_file in enumerate(trace_files):
        try:
            with open(trace_file, 'r') as f:
                data = json.load(f)
            
            if isinstance(data, dict):
                events = data.get("traceEvents", [])
                if idx == 0: metadata = {k: v for k, v in data.items() if k != "traceEvents"}
            else:
                events = data
            
            # --- ROBUST PID/TID OFFSETTING ---
            for event in events:
                if "pid" in event:
                    try: event["pid"] = int(event["pid"]) + (idx * 100000)
                    except ValueError: event["pid"] = f"{event['pid']}_{idx}"

                if "tid" in event:
                    try: event["tid"] = int(event["tid"]) + (idx * 100000)
                    except ValueError: event["tid"] = f"{event['tid']}_{idx}"
                
                if "name" in event and isinstance(event["name"], str):
                    if not event["name"].startswith("[GPU"):
                        event["name"] = f"[GPU {idx}] {event['name']}"
            
            merged_events.extend(events)
            print(f"    Added {len(events)} events from GPU {idx}")
        except Exception as e:
            print(f"    Warning: Could not read {trace_file}: {e}")

    merged_trace = metadata.copy()
    merged_trace["traceEvents"] = merged_events
    
    with open(MERGED_TRACE_FILE, 'w') as f:
        json.dump(merged_trace, f)
    
    print(f"\n MERGE COMPLETE!")
    print(f" Saved to: {MERGED_TRACE_FILE}")
    print(f" Upload this file to https://ui.perfetto.dev/\n")

# --- Training Configuration ---
device = model_engine.device
num_epochs = 1
start_time = time.time()
PROFILING_STEPS = 0

if rank == 0: 
    print(f"\n{'='*70}")
    print(f"--- STARTING TRAINING WITH PROFILING ---")
    print(f"  - Profiling: ALL ranks (first {PROFILING_STEPS} steps)")
    print(f"  - Training: Continues FULLY after profiling")
    print(f"{'='*70}\n")

try:
    # --- PROFILER INITIALIZATION (ALL RANKS) ---
    prof = torch.profiler.profile(
        activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
        schedule=torch.profiler.schedule(wait=1, warmup=2, active=5, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler(TRACE_DIR),
        record_shapes=True,
        profile_memory=True,
        with_stack=True,
        with_flops=True
    )
    # prof.start()
    
    model_engine.train()
    
    for epoch in range(num_epochs):
        train_sampler.set_epoch(epoch)
        epoch_loss = 0.0
        num_batches = 0
        
        for i, batch in enumerate(train_loader):
            
            # --- Profiling Window ---
            is_profiling = (i < PROFILING_STEPS)
            prof_context = None
            
            if is_profiling:
                prof_context = torch.profiler.record_function(f"## [GPU {rank}] Step {i} ##")
                prof_context.__enter__()
            
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward
            if is_profiling:
                with torch.profiler.record_function(f"## [GPU {rank}] FWD ##"):
                    outputs = model_engine(inputs, labels=labels)
                    loss = outputs.loss
            else:
                outputs = model_engine(inputs, labels=labels)
                loss = outputs.loss

            # Backward
            if is_profiling:
                with torch.profiler.record_function(f"## [GPU {rank}] BWD ##"):
                    model_engine.backward(loss)
            else:
                model_engine.backward(loss)

            # Step
            if is_profiling:
                with torch.profiler.record_function(f"## [GPU {rank}] OPT ##"):
                    model_engine.step()
            else:
                model_engine.step()

            # Metrics
            loss_val = loss.item()
            epoch_loss += loss_val
            num_batches += 1

            if rank == 0:
                wandb.log({"loss": loss_val, "step": i})
                if i % 10 == 0:
                    print(f"  [GPU {rank}] Epoch {epoch} | Step {i} | Loss: {loss_val:.4f}")

            # --- Profiler Stepping ---
            if is_profiling:
                prof.step()
                if prof_context: prof_context.__exit__(None, None, None)
            
            # --- STOP & MERGE (But continue training) ---
            if i == PROFILING_STEPS:
                prof.stop() # Save trace files to disk
                
                # 1. Sync all ranks to ensure writing is done
                if world_size > 1: dist.barrier()
                
                # 2. Rank 0 performs the merge immediately
                if rank == 0:
                    print(f"\n>>> Profiling complete. Merging traces while training continues... <<<")
                    merge_traces_logic()
                    print(">>> Resuming full training loop... <<<\n")

    # End of Epoch
    if rank == 0:
        print(f"\n{'='*70}")
        print(f"TRAINING COMPLETE")
        print(f"{'='*70}\n")
        wandb.finish()

except Exception as e:
    print(f"[Rank {rank}] ERROR: {e}")
    import traceback
    traceback.print_exc()
finally:
    if world_size > 1:
        dist.barrier()
        dist.destroy_process_group()
    print(f"--- [Rank {rank}] Done ---")

Writing train_deepspeed_zeropp.py


In [24]:
!deepspeed --num_gpus=2 train_deepspeed_zeropp.py

2025-12-07 01:24:17.583336: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765070657.606290    1036 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765070657.613769    1036 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
[2025-12-07 01:24:22,734] [WARNING] [runner.py:232:fetch_hostfile] Unable to 

In [25]:
import os
os.system('zip -r deepspeed_zeropp_trace.zip /kaggle/working/pytorch_traces/deepspeed_zeropp_trace')

  adding: kaggle/working/pytorch_traces/deepspeed_zeropp_trace/

0

 (stored 0%)
